In [2]:
!wget https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/master/advanced_functionality/tensorflow_bring_your_own/utils/generate_cifar10_tfrecords.py
!pip install ipywidgets
!python generate_cifar10_tfrecords.py --data-dir cifar10

--2022-01-13 00:09:52--  https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/master/advanced_functionality/tensorflow_bring_your_own/utils/generate_cifar10_tfrecords.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4986 (4.9K) [text/plain]
Saving to: ‘generate_cifar10_tfrecords.py.1’

generate_cifar10_tf 100%[===================>]   4.87K  --.-KB/s    in 0s      

2022-01-13 00:09:52 (66.6 MB/s) - ‘generate_cifar10_tfrecords.py.1’ saved [4986/4986]

  Using cached ipywidgets-7.6.5-py2.py3-none-any.whl (121 kB)
  Using cached widgetsnbextension-3.5.2-py2.py3-none-any.whl (1.6 MB)
  Using cached nbformat-5.1.3-py3-none-any.whl (178 kB)
  Using cached jupyterlab_widgets-1.0.2-py3-none-any.whl (243 kB)
  Using cached jsonschema-3.2.0-py2.py3-none-a

In [3]:
import time, os, sys
import sagemaker, boto3
import numpy as np
import pandas as pd

sess = boto3.Session()
sm   = sess.client('sagemaker')
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)

datasets = sagemaker_session.upload_data(path='cifar10', key_prefix='datasets/cifar10-dataset')
datasets 

's3://sagemaker-us-west-1-821921608777/datasets/cifar10-dataset'

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

training_experiment = Experiment.create(
                                experiment_name = "sagemaker-training-experiments-3", 
                                description     = "Experiment to track cifar10 training trials", 
                                sagemaker_boto_client=sm)

In [4]:
import horovod.tensorflow.keras as hvd
import tensorflow.keras.backend as K

In [5]:
hvd_instance_type = 'ml.p3.16xlarge'
hvd_instance_count = 2
hvd_processes_per_host = 1

print(f'Distributed training with a total of {hvd_processes_per_host*hvd_instance_count} workers: {hvd_instance_count} instances of {hvd_instance_type}')
print(f'{hvd_processes_per_host} GPU(s) per instance')

Distributed training with a total of 2 workers: 2 instances of ml.p3.16xlarge
1 GPU(s) per instance


In [6]:
distributions = {
                 'mpi': {
                          'enabled'           : True,
                          'processes_per_host': hvd_processes_per_host,
                          'custom_mpi_options': '-verbose --NCCL_DEBUG=INFO -x OMPI_MCA_btl_vader_single_copy_mechanism=none'
                        }
                }

In [7]:
from sagemaker.tensorflow import TensorFlow

hyperparams={'epochs'       : 30,
             'learning-rate': 0.01,
             'batch-size'   : 256,
             'weight-decay' : 2e-4,
             'momentum'     : 0.9,
             'optimizer'    : 'adam'}

bucket_name = sagemaker_session.default_bucket()
output_path = f's3://{bucket_name}/jobs'
metric_definitions = [{'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'}]

tf_estimator = TensorFlow(entry_point          = 'cifar10-tf-horovod-sagemaker.py', 
                          output_path          = f'{output_path}/',
                          code_location        = output_path,
                          role                 = role,
                          train_instance_count = hvd_instance_count, 
                          train_instance_type  = 'ml.g4dn.xlarge',
                          train_volume_size    = 50,
                          framework_version    = '1.15.2', 
                          py_version           = 'py3',
                          script_mode          = True,
                          metric_definitions   = metric_definitions,
                          sagemaker_session    = sagemaker_session,
                          hyperparameters      = hyperparams,
                          distributions        = distributions)

job_name=f'tensorflow-single-gpu-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'
tf_estimator.fit({'training'  : datasets,
                  'validation': datasets,
                  'eval'      : datasets},
                 job_name = job_name,
                 experiment_config=experiment_config)

distributions has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


NameError: name 'experiment_config' is not defined